# MP Expenses Project v2.0
by Darren Christie 
Created October 2020

This is a project notebook that looks at MP Expenses. This version of the project uses a sqlite3 database instead of
a csv file.
The notebook compares a single MP (which will probably be your local MP) with the expenses of all MPs.

## The Data
Data has been obtained from the [IPSA website](https://www.theipsa.org.uk/mp-costs/annual-publication/) starting from 2010/2011 csv files were downloaded for individual claims for each reported year. The datasets were downloaded 6/6/2020.
These csv files can be found in the data/raw folder.

The following awk command was used to merge the individual csv files into a combined csv file with a single header.

`awk '(NR == 1) || (FNR > 1)' Individual*.csv > combined_claims.csv`

FNR represents the number of the processed record in a single file. NR represents it globally. Therefore the first line is accepted and the rest are ignored.
I can not take credit for the above awk command. I got it from the StackExchange website (accessed on 14/5/2020) and was an answer provided by a Marek Grac.

The combined csv file can be found in the data/processed folder.
This combined csv file was then imported into a sqlite3 database.

In [1]:
# our standard import for our projects
import warnings
warnings.simplefilter('ignore', FutureWarning)

import matplotlib
import matplotlib.pyplot as plot
matplotlib.rcParams['axes.grid'] = True # show gridlines by default

# tells Jupyter to display all charts inside this notebook, immediately after each call to plot()
%matplotlib inline

import datetime as dt
import numpy as np
import sqlite3 as lite

from pandas import *

In [2]:
# suppress scientific notation globally
# taken from https://stackoverflow.com/questions/21137150/format-suppress-scientific-notation-from-python-pandas-aggregation-results
pandas.options.display.float_format = '{:.2f}'.format

### function definitions used elsewhere in the notebook

In [3]:
# a function to calculate the range
# this works with a groupby function call
# a modified version of code found at http://www.pybloggers.com/2018/12/python-pandas-groupby-tutorial/
def stat_range(df):
    rang = df.max() - df.min()
    
    return rang

## Clean up data
This step has been moved further down once we have got the results back from the sql

## Assumptions about the data
* that the MPs expenses year follows a tax year and runs from 1st April - 31st March.
* that -ve values in the Amount Paid and Claimed columns means that the MP has had to pay money back. This is currently a query I have raised with the IPSA via social media to confirm one way or the other.

## Processing

### Set the MP we are looking at
The MP that we are interested in investigating their expenses.
If you are unsure who your MP is you can find out at [FindYourMP](https://members.parliament.uk/FindYourMP). Enter your post code and it will tell you who you MP is.
**NOTE:** The name of your MP needs to match exactly as it appears in the csv file/dataframe. Otherwise it will not find anything.

In [4]:
LOCALMP = "Stephen Barclay"

### Set some other constants that we will use throughout the notebook

In [5]:
STARTTAXYEAR = 2010
ENDTAXYEAR = 2020

### Create the connection to our database

In [6]:
sqlCon = lite.connect('data/processed/mpexpenses.db')

### Build our sql query to retrieve all of the local MPs data from the database

In [7]:
sqlQuery = f"select \"Date\",\"Category\",\"Expense Type\",\"Amount Paid\" from expenses where \"MP's Name\" = \"{LOCALMP}\""
print (sqlQuery)

select "Date","Category","Expense Type","Amount Paid" from expenses where "MP's Name" = "Stephen Barclay"


### Execute our query and get the results into a dataframe

In [8]:
df = pandas.read_sql_query(sqlQuery,sqlCon)

### Set the data type of a couple of the columns and create an index for the dataframe

In [9]:
# correct column types
df['Date'] = to_datetime(df['Date'])
df['Amount Paid'] = to_numeric(df['Amount Paid'])
# set the Date column to our index
df.index = df['Date']
df = df.sort_index()

In [10]:
# These next line of code basically removes -ve values, which I have assumed means that the MP has repaid money 
# to the IPSA.
newdf = df[df['Amount Paid'] > 0]
newdf

,Date,Category,Expense Type,Amount Paid
Date,,,,
2010-01-09,2010-01-09,MP Travel,Own Car MP,3.08
2010-01-09,2010-01-09,MP Travel,Own Car MP,39.28
2010-01-09,2010-01-09,MP Travel,Own Car MP,36.48
2010-01-09,2010-01-09,Miscellaneous Expenses,Contingency,204.16
2010-01-09,2010-01-09,Accommodation,Accommodation Rent,1450.00
...,...,...,...,...
2019-12-03,2019-12-03,Staffing,Own Vehicle Car Volunteer,4.86
2019-12-03,2019-12-03,Staffing,Own Vehicle Car Volunteer,4.86
2019-12-08,2019-12-08,Office Costs,Equipment - purchase,8.90


### Start to produce some analysis based on the data retrieved

### Total expenses claimed for all time

In [11]:
print (f"Between 1st April {STARTTAXYEAR} and 31st March {ENDTAXYEAR} MP {LOCALMP} claimed £{newdf['Amount Paid'].sum()} in expenses.")

Between 1st April 2010 and 31st March 2020 MP Stephen Barclay claimed £1445649.29 in expenses.


In [12]:
print ("This is a breakdown of that figure by category claimed between those dates.")
newdf.groupby('Category')['Amount Paid'].sum()

This is a breakdown of that figure by category claimed between those dates.


Category
Accommodation               146634.07
Dependant Travel              3051.99
MP Travel                    70831.82
Miscellaneous Expenses        2910.68
Office Costs                201886.41
Office Costs Expenditure       837.60
Staff Travel                 12307.91
Staffing                   1007188.81
Name: Amount Paid, dtype: float64

In [13]:
localMPYearlyDetail = DataFrame()
localMPYearlySummary = DataFrame()
currTaxYear = STARTTAXYEAR

# loop round and extract each tax years summary data at the two levels we are interested in
for counter in range (0,(ENDTAXYEAR - STARTTAXYEAR)):
    
    # create our tax year index i.e. 2010/2011
    tempIndex = str(currTaxYear)+'/'+str(currTaxYear+1)
    
    # extract the data from the dataframe that falls in the current tax year
    tempDF = newdf.loc[dt.datetime(currTaxYear,4,1):dt.datetime(currTaxYear+1,3,31)]
    
    # generate our summary stats based on the category
    yearlyCategorySummary = tempDF.groupby('Category')['Amount Paid'].agg(['sum','mean', 'median', 'max', 'min',stat_range,'std'])
    yearlyCategorySummary['Tax Year'] = tempIndex # add the tax year as a column to the dataframe
    localMPYearlySummary = localMPYearlySummary.append(yearlyCategorySummary) # append the stats we generated to the dataframe
    
    # generate our detailed stats based on the category and the expense type within eacg category
    yearlyDetailSummary = tempDF.groupby(['Category','Expense Type'])['Amount Paid'].agg(['sum','mean', 'median', 'max', 'min',stat_range,'std'])
    yearlyDetailSummary['Tax Year'] = tempIndex # add the tax year as a column to the dataframe
    localMPYearlyDetail = localMPYearlyDetail.append(yearlyDetailSummary) # append the stats we generated to the dataframe
    
    currTaxYear += 1 # move to next tax year
    

# turn into a multiindex dataframe
localMPYearlySummary.reset_index(level=0, inplace=True)
#localMPYearlySummary.set_index(['Tax Year','Category'],inplace=True)
localMPYearlySummary.set_index(['Category','Tax Year'],inplace=True)
localMPYearlySummary.sort_index(inplace=True)

localMPYearlyDetail.reset_index(level=[0,1], inplace=True)
localMPYearlyDetail.set_index(['Category','Expense Type','Tax Year'],inplace=True)
localMPYearlyDetail.sort_index(inplace=True)

localMPYearlyDetail = localMPYearlyDetail.round(decimals=2) # round to 2 decimal places
localMPYearlySummary = localMPYearlySummary.round(decimals=2) # round to 2 decimal places

In [14]:
localMPYearlyDetail

sum    mean  \
Category      Expense Type                         Tax Year                     
Accommodation Accommodation Rent                   2010/2011 15001.15 1071.51   
                                                   2011/2012 25004.94 1562.81   
                                                   2012/2013 22228.35 1852.36   
                                                   2013/2014 13866.68 1733.34   
              Council Tax                          2010/2011   181.00  181.00   
...                                                               ...     ...   
Staffing      Staff Training Costs                 2015/2016   816.00  816.00   
                                                   2018/2019   420.00  420.00   
              Training - staff                     2019/2020   438.60  438.60   
              Volunteer - agreed arrangement costs 2018/2019   116.91    4.87   
                                                   2019/2020   380.34    5.76   

                                                              median     max  \
Category      Expense Type                         Tax Year                    
Accommodation Accommodation Rent                   2010/2011 1450.00 1450.00   
                                                   2011/2012 1860.21 2790.00   
                                                   2012/2013 1860.00 1860.00   
                                                   2013/2014 1733.34 1841.67   
              Council Tax                          2010/2011  181.00  181.00   
...                                                              ...     ...   
Staffing      Staff Training Costs                 2015/2016  816.00  816.00   
                                                   2018/2019  420.00  420.00   
              Training - staff                     2019/2020  438.60  438.60   
              Volunteer - agreed arrangement costs 2018/2019    4.86    4.95   
                                                   2019/2020    4.86   10.80   

                                                                 min  \
Category      Expense Type                         Tax Year            
Accommodation Accommodation Rent                   2010/2011  202.00   
                                                   2011/2012  100.00   
                                                   2012/2013 1841.67   
                                                   2013/2014 1625.00   
              Council Tax                          2010/2011  181.00   
...                                                              ...   
Staffing      Staff Training Costs                 2015/2016  816.00   
                                                   2018/2019  420.00   
              Training - staff                     2019/2020  438.60   
              Volunteer - agreed arrangement costs 2018/2019    4.86   
                                                   2019/2020    4.68   

                                                              stat_range  \
Category      Expense Type                         Tax Year                
Accommodation Accommodation Rent                   2010/2011     1248.00   
                                                   2011/2012     2690.00   
                                                   2012/2013       18.33   
                                                   2013/2014      216.67   
              Council Tax                          2010/2011        0.00   
...                                                                  ...   
Staffing      Staff Training Costs                 2015/2016        0.00   
                                                   2018/2019        0.00   
              Training - staff                     2019/2020        0.00   
              Volunteer - agreed arrangement costs 2018/2019        0.09   
                                                   2019/2020        6.12   

                                             